In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import talib as ta
import plotly.express as px
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

np.random.seed(42)

In [ ]:
# Get the data for the stock
start_date = '2012-01-01'
end_train = '2017-12-31'
end_date = '2023-12-31'
ticker = 'AAPL'

df = yf.download(ticker, start=start_date, end=end_date)

In [ ]:
df["Adj Low"] = df["Low"] - (df["Close"]-df["Adj Close"])
df["Adj High"] = df["High"] - (df["Close"]-df["Adj Close"])

In [ ]:
df["Returns"] = df["Adj Close"].pct_change()

df["Target"] = df["Returns"].shift(-1)
df["Target_cat"] = np.where(df["Target"] > 0, 1, 0)

In [ ]:
df["std15"] = df["Adj Close"].rolling(15).std()
df["moving_average"] = df["Adj Close"].rolling(15).mean()

df["zscore"] = (df["Adj Close"]-df["moving_average"])/df["std15"]
df["aroon"] = ta.AROONOSC(df["Adj High"], df["Adj Low"], timeperiod=14)
df["price_trend"] = df["Returns"].shift().rolling(4).sum()

In [ ]:
df["zscore"] = pd.qcut(df["zscore"], 6, labels = False)
df["aroon"] = pd.qcut(df["aroon"], 4, labels = False)
df["price_trend"] = pd.qcut(df["price_trend"], 6, labels = False)

In [ ]:
df = df.dropna()
features = ["aroon", "zscore", "price_trend"]

In [ ]:
X_train, X_test = df[features].loc[start_date:end_train], df[features].loc[end_train:end_date]
y_train, y_test = df["Target_cat"].loc[start_date:end_train], df["Target_cat"].loc[end_train:end_date]

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

---
---
BACKTESTING

In [ ]:
df["train_test"] = np.where(df.index > end_train, "Test", "Train")

y_pred_all = model.predict(df[features])
df["Signal"] = np.where(y_pred_all == 1, 1, -1)

df["Model_Returns"] = df["Signal"] * df["Target"]

In [ ]:
fig = px.line(df, x=df.index, y=df["Model_Returns"].cumsum()*100, color="train_test",
              labels={"y": "Cumulative Returns (%)"},
              title=f"{model.__class__.__name__} - {ticker}",
              line_shape="linear")

fig.show()